In [2]:
"""
标准库中的所有映射类型都是利用dict来实现的，因此他们有共同的限制，只有可散列的数据类型才能用做这些映射里的键。
什么是可散列类型呢？Python官方给出的翻译是：如果一个对象是可散列的，那么在这个对象的生命周期中，他的散列值是不会变的
（它需要实现__hash__()方法）。它可以与其他对象作比较（还需实现__eq__()方法）。如果一个可散列对象与另一个可散列对象
是相等的，那么他们的散列值hash value一定是相等的。

原子不可变数据类型（str，bytes和数值类型）都是hashable类型，frozenset也是hashable的，因为根据其定义，frozenset里只可容纳可散列类型。
元组也是hashable的，但只有当元组包含的所有元素都是hashable类型的情况下它才是可散列的。

list等可变对象是不可散列的，因为随着数据的改变他们的哈希值会变化导致进入错误的哈希表。

一般用户自定义的类型的对象都是可散列的，散列值就是它们的id()函数的返回值，所以所有这些对象在比较的时候都是不想等的。
如果一个对象实现了__eq__()方法，并且在方法中用到了这个对象的内部状态的话，那么只有当所有这些内部状态都是不可变的情
况下，这个对象才是可散列的。

"""
tt = (1,2,(30,40))
print(hash(tt))

t1 = (1,2,[30,40])
print(hash(t1))

8027212646858338501


TypeError: unhashable type: 'list'

In [8]:
tf = (1,2,frozenset([30,40]))
print(hash(tf))

-4118419923444501110


In [9]:
"""
字典的生成方式，下面的五种方式得到的结果是一样的
"""
a = dict(one=1,two=2,three=3)
b = {'one':1,'two':2,'three':3}
c = dict(zip(['one','two','three'],[1,2,3]))
d = dict([('two',2),('one',1),('three',3)])
e = dict({'three':3,'one':1,'two':2})
print(a==b==c==d==e)

True


In [11]:
# 除此之外，我们还可以使用字典推导式
DIAL_CODES = [
    (86,'China'),
    (91,'India'),
    (1,'United States'),
    (62,'Indonesia'),
    (55,'Brazil'),
    (92,'Pakistan'),
    (880,'Bangladesh')
]
country_code = {country:code for code,country in DIAL_CODES}
print(country_code)
country_code1 = {code:country.upper() for country,code in country_code.items()}
print(country_code1)

{'Bangladesh': 880, 'China': 86, 'Indonesia': 62, 'Pakistan': 92, 'Brazil': 55, 'United States': 1, 'India': 91}
{880: 'BANGLADESH', 1: 'UNITED STATES', 86: 'CHINA', 55: 'BRAZIL', 91: 'INDIA', 92: 'PAKISTAN', 62: 'INDONESIA'}


In [12]:
"""字典处理找不到的键"""
"""
字典取值的时候，会调用__getitem__方法，但是有时候会碰到找不到的键，该如何处理呢？
有下面几种方法：
1、get(key,default)
2、dict.setdefault
3、collections.defaultdict
4、__missing__方法
"""

'\n有下面几种方法：\n1、get(key,default)\n2、dict.setdefault\n3、collections.defaultdict\n4、__missing__方法\n'

In [13]:
"""
1、get(key,default)，并不是一个很好的解决方案，效率偏低，我们看代码
"""
points_game = {'Harden':[58,57,48],'Curry':[20,10]}
new_games = [('Harden',61),('Durant',20)]
for name,point in new_games:
    point_his = points_game.get(name,[]) # 一次查询操作，判断name有没有出现在字典中
    point_his.append(point) # 添加新的得分纪录
    points_game[name] = point_his # 将得分纪录和name添加到字典中，又涉及一次插入操作
print(points_game)

{'Durant': [20], 'Harden': [58, 57, 48, 61], 'Curry': [20, 10]}


In [15]:
"""
2、dict.setdefault
"""
points_game = {'Harden':[58,57,48],'Curry':[20,10]}
new_games = [('Harden',61),('Durant',20)]
for name,point in new_games:
    # 如果名字没有出现，则把名字和空列表放进映射，然后返回这个空列表，这样就能在不进行第二次查找的情况下更新列表了
    points_game.setdefault(name,[]).append(point) 
print(points_game)

{'Durant': [20], 'Harden': [58, 57, 48, 61], 'Curry': [20, 10]}


In [17]:
"""
3、collections.defaultdict
在实例化一个defaultdict的时候，需要给构造方法提供一个可调用的对象，这个可调用对象会在__getitem__碰到找不到的键的时候被调用。
比如下面的例子，在找不到name的情况下，会调用list()来建立一个新列表，并赋值给points_game[name],继而被当作返回值返回。
"""
import collections
points_game = collections.defaultdict(list)
points_game['Harden'] = [58,57,48]
points_game['Curry'] = [20,10]
new_games = [('Harden',61),('Durant',20)]
for name,point in new_games:
    points_game[name].append(point)
print(points_game)

defaultdict(<class 'list'>, {'Durant': [20], 'Harden': [58, 57, 48, 61], 'Curry': [20, 10]})


In [23]:
"""
4、__missing__方法
所有映射类型在处理找不到键的时候，都会牵扯__missing__方法。虽然基类dict没有实现这个方法，但是dict知道这个东西是存在的。
也就是说，如果一个类继承了dict，然后这个继承类提供了__missing__方法，那么在__getitem__碰到找不到键的时候，Python就会
自动调用它。
__missing__方法只会被__getitem__调用。
"""
class strKeyDict0(dict):
    def __missing__(self,key):
        if isinstance(key,str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self,key,default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    # 注意这里的一个实现细节，我们没有调用key in self,而是key in self.keys()。因为key in self本身调用的还是__contains__方法，
    # 这样会造成递归调用。
    def __contains__(self,key):
        return key in self.keys() or str(key) in self.keys()
    
d = strKeyDict0([('2','two'),('4','four')])
print(d['2'])
# 4不在我们的键中，会调用我们的__missing__方法，发现'4'在字典中，则返回'4'对应的值。
print(d[4])
print(d[1])

two
four


KeyError: '1'

In [26]:
print(d.get('2'))
print(d.get(4))
print(d.get(1,'N/A'))

two
four
N/A


In [25]:
print(2 in d)
print(1 in d)

True
False


In [29]:
"""字典的变种"""
"""
1、collections.OrderedDict，在添加键的时候会保持顺序，因此键的迭代次序总是一致的。在使用poiitem方法时，会默认删除最后一个元素，
但是如果popitem(last=False)这样调用，删除并返回第一个添加的元素。

下面的代码中，OrderedDict的输出顺序喝插入顺序一致，但是普通的dict却不是。
"""
from collections import OrderedDict
orderdict = OrderedDict()
orderdict['A'] = 1
orderdict['B'] = 2
orderdict['C'] = 3

from collections import OrderedDict
commendict = dict()
commendict['A'] = 1
commendict['B'] = 2
commendict['C'] = 3

for key,value in orderdict.items():
    print(key,value)

for key,value in commendict.items():
    print(key,value)


A 1
B 2
C 3
A 1
C 3
B 2


In [31]:
"""
2、collections.chainMap
可以容纳数个不同的映射对象，在进行键查找操作时，会被当作一个整体进行逐个查找，直到被找到为止
"""
from collections import ChainMap
dicta = {'A':1,'B':2}
dictb = {'A':5,'C':7}
chainmap = ChainMap(dicta,dictb)
print(chainmap['A'])
print(chainmap['C'])

1
7


In [36]:
"""
3、collections.Counter
用来计数，实现了+和-运算符来合并记录，还有most_common这类很有用的方法
"""
from collections import Counter
ct = collections.Counter('abracadabra')
print(ct)
ct1 = collections.Counter('abadewetdrr')
print(ct1)
print(ct + ct1)
# 从结果可以看到，若ct中有，ct1中没有，则保留ct中的记录，如c
# 若ct和ct1中都有，ct的数量小于等于ct1中的数量，则结果中没有该记录，如d
# 否则，结果中的数是二数量的差值，如a，b
print(ct - ct1) 
# 就地更新，返回值是None
ct.update('aaaaazzz')
print(ct)

Counter({'a': 5, 'r': 2, 'b': 2, 'c': 1, 'd': 1})
Counter({'a': 2, 'r': 2, 'e': 2, 'd': 2, 'w': 1, 't': 1, 'b': 1})
Counter({'a': 7, 'r': 4, 'd': 3, 'b': 3, 'e': 2, 'c': 1, 'w': 1, 't': 1})
Counter({'a': 3, 'c': 1, 'b': 1})
Counter({'a': 10, 'z': 3, 'r': 2, 'b': 2, 'c': 1, 'd': 1})


In [39]:
"""
4、collections.UserDict
创造自定义映射类型，以UserDict为基类，更加方便。因为dict在某些方法的实现上会走一些捷径，导致我们不得不在子类中重写这些方法，
而UserDict就没有这些问题。

UserDict并不是dict的子类，但是其中的data属性，是dict的实例，这个属性实际上就是UserDict最终存储数据的地方。因此，我们可以让__setitem__
和__contains__里的代码更简洁
"""
class strKeyDict(dict):
    def __missing__(self,key):
        if isinstance(key,str):
            raise KeyError(key)
        return self[str(key)]
    # 这里无需使用str(key) in self.data.keys()
    def __contains__(self,key):
        return str(key) in self.data
    
    def __setitem__(self,key,item):
        self.data[str(key)] = item

In [41]:
"""
不可变映射类型：types模块中引入了一个封装类名叫MappingProxyType。
如果给这个类一个映射，它会返回一个只读的映射视图。
如果原映射做了修改，我们可以通过这个视图观察到，但是无法通过修改这个视图对原映射做出修改
"""
from types import MappingProxyType
d = {1:'A'}
d_proxy = MappingProxyType(d)
print(d_proxy)
print(d_proxy[1])
d[2] = 'B'
print(d_proxy)
d_proxy[3] = 'C'

{1: 'A'}
A
{1: 'A', 2: 'B'}


TypeError: 'mappingproxy' object does not support item assignment